<a href="https://colab.research.google.com/github/sourav044/Python/blob/master/NLP-HandsON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity
text = ['human', 'interface', 'computer']
model = Word2Vec(common_texts, size=20, min_count=1)  # train word-vectors

index = WmdSimilarity(text, model)
# Make query.
query = ['trees']
sims = index[query]


In [20]:
print(common_texts)
print(model)
print(sims)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]
Word2Vec(vocab=12, size=20, alpha=0.025)
[0. 0. 0.]


In [21]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity

model = Word2Vec(common_texts, size=20, min_count=1)  # train word-vectors
index = WmdSimilarity(common_texts, model)
 # Make query.
query = ['human', 'interface', 'computer']
sims = index[query]
print(common_texts)
print(sims)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]
[1.         0.48811893 0.49534619 0.47371859 0.4654382  0.41035556
 0.42039188 0.43218044 0.42965077]


NOW starting the debug 

In [22]:
pip install PyPDF2

In [0]:
import nltk
from nltk.tokenize import sent_tokenize

In [24]:
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
import spacy.cli
lang = spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
from collections import Counter
import nltk
from nltk.tokenize import sent_tokenize
from keras.preprocessing.text import Tokenizer 
import spacy
import PyPDF2
import keras
import tensorflow as tf
import gensim
from nltk.corpus import stopwords

In [0]:

def combine(sub_keys, keywords_splits, lb, mb, ub):
    whitespace = ' '
    while mb != ub:
        keywords_splits.append(whitespace.join(sub_keys[lb: mb]))
        keywords_splits.append(whitespace.join(sub_keys[mb: ub]))
        mb += 1
    del sub_keys[0]
    if len(sub_keys) > 2:
        combine(sub_keys, keywords_splits, 0, 1, len(sub_keys))


def keywords_splitter(keywords, keywords_splits):

    for key in keywords:
        sub_keys = key.split()

        if len(sub_keys) > 2:
            combine(sub_keys, keywords_splits, 0, 1, len(sub_keys))


def pre_query(question_query):

    keywords = question_query.get_features()

    keywords = [feat.lower() for feat in keywords]
    whitespace = ' '
    keywords_splits = whitespace.join(keywords).split()

    keywords_splitter(keywords, keywords_splits)
    keywords_splits = list(set(keywords_splits + keywords))

    return keywords_splits


def query2vec(query, dictionary):

    print("Searching: {0}".format(query))
    corpus = dictionary.doc2bow(query)

    return corpus


def doc2vec(documents):
         
    texts = [[word for word in doc.lemma_.split() if word not in ['a','b']]for doc in documents]

    frequency = Counter()
    for sent in texts:
        for token in sent:
            frequency[token] += 1

    
    text_token = [[token for token in snipp]for snipp in texts]
    
   
    texts = [[token for token in snipp if frequency[token] > -1]for snipp in texts]

    dictionary = gensim.corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(snipp) for snipp in texts]

    return corpus, dictionary, texts, text_token


def transform_vec(corpus, query_corpus):
    lsidf = gensim.models.LsiModel(corpus)

    corpus_lsidf = lsidf[corpus]
    query_lsidf = lsidf[query_corpus]

    return corpus_lsidf, query_lsidf


def similarity(corpus_lsidf, query_lsidf):
    index = gensim.similarities.SparseMatrixSimilarity(corpus_lsidf, num_features=200000)

    simi = index[query_lsidf]

    simi_sorted = sorted(enumerate(simi), key=lambda item: -item[1])
    return simi_sorted




In [28]:
text = "My name is Sourav and I am from India. My age is 24. I am currently stying in Passau. I am studing Mobile and embedded system. My father name is Uttam Kumar gupta. He is a business man. He own a shop. His work is of Photography. "
en_doc = nlp(u'' + text)
sentences = list(en_doc.sents)
print(sentences)


[My name is Sourav and I am from India., My age is 24., I am currently stying in Passau., I am studing Mobile and embedded system., My father name is Uttam Kumar gupta., He is a business man., He own a shop., His work is of Photography.]


### **1ST Remove Stop Word and Bag of word**

In [0]:
corpus, dictionary, text, token = doc2vec(sentences)

In [30]:
print(text)
print(token)
print(dictionary)

[['-PRON-', 'name', 'be', 'Sourav', 'and', '-PRON-', 'be', 'from', 'India', '.'], ['-PRON-', 'age', 'be', '24', '.'], ['-PRON-', 'be', 'currently', 'stye', 'in', 'Passau', '.'], ['-PRON-', 'be', 'stud', 'Mobile', 'and', 'embedded', 'system', '.'], ['-PRON-', 'father', 'name', 'be', 'Uttam', 'Kumar', 'gupta', '.'], ['-PRON-', 'be', 'business', 'man', '.'], ['-PRON-', 'own', 'shop', '.'], ['-PRON-', 'work', 'be', 'of', 'Photography', '.']]
[['-PRON-', 'name', 'be', 'Sourav', 'and', '-PRON-', 'be', 'from', 'India', '.'], ['-PRON-', 'age', 'be', '24', '.'], ['-PRON-', 'be', 'currently', 'stye', 'in', 'Passau', '.'], ['-PRON-', 'be', 'stud', 'Mobile', 'and', 'embedded', 'system', '.'], ['-PRON-', 'father', 'name', 'be', 'Uttam', 'Kumar', 'gupta', '.'], ['-PRON-', 'be', 'business', 'man', '.'], ['-PRON-', 'own', 'shop', '.'], ['-PRON-', 'work', 'be', 'of', 'Photography', '.']]
Dictionary(29 unique tokens: ['-PRON-', '.', 'India', 'Sourav', 'and']...)


In [31]:
print(corpus)

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1)], [(0, 1), (1, 1), (5, 1), (8, 1), (9, 1)], [(0, 1), (1, 1), (5, 1), (10, 1), (11, 1), (12, 1), (13, 1)], [(0, 1), (1, 1), (4, 1), (5, 1), (14, 1), (15, 1), (16, 1), (17, 1)], [(0, 1), (1, 1), (5, 1), (7, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(0, 1), (1, 1), (5, 1), (22, 1), (23, 1)], [(0, 1), (1, 1), (24, 1), (25, 1)], [(0, 1), (1, 1), (5, 1), (26, 1), (27, 1), (28, 1)]]


### **Feature Extract**

In [0]:

def get_detail(sentence):
    for token in sentence:
        print("{0} -- Lemma:{1}, Tag:{2}, EntType:{3}, Dep:{4}, Head:{5}"
                     .format(token.text, token.lemma_, token.tag_, token.ent_type_, token.dep_, token.head))


def get_compound_nouns(en_doc, token, token_text):

    """
    Recursively find the left and right compound nouns
    """

    parent_token = token

    print("Compound Noun:{0} DEP {1}".format(token.text, token.dep_))

    # If previous token is a compound noun
    while token.i > 0 and en_doc[token.i - 1].dep_ == "compound":
        token_text = en_doc[token.i - 1].text + " " + token_text
        token = en_doc[token.i - 1]
        # if the compound noun has any adjective modifier
        token_text = get_adj_phrase(token, token_text)

    token = parent_token

    # If next token is a compound noun
    while token.i < len(en_doc) - 1 and en_doc[token.i + 1].dep_ == "compound":
        token_text = token_text + " " + en_doc[token.i + 1].text
        token = en_doc[token.i + 1]
        # if the compound noun has any adjective modifier
        token_text = get_adj_phrase(token, token_text)

    # NOTE: Can token.shape_ == Xxxx... or XXXX... token.ent_iob_ help us here ...?

    return token_text

def this_is_adjective(dep_):
    return dep_ == "amod" or dep_ == "acomp" or dep_ == "ccomp"

def get_adj_phrase(token, token_text):

    """
    To fetch all the adjectives describing the noun
    """

    # amod: An adjectival modifier of a noun is any adjectival phrase that serves to modify the meaning of the noun.
    # ccomp: A clausal complement of a verb or adjective is a dependent clause which is a core argument.
    #        That is, it functions like an object of the verb, or adjective.
    # acomp: An adjectival complement of a verb is an adjectival phrase which functions as the complement

    for child in token.children:
        if(this_is_adjective(child.dep_)) and child.text != "much" and child.text != "many":
            # if child.dep_ == "amod" or child.dep_ == "acomp" or child.dep_ == "ccomp":  # not for how many
            #     if child.text != "much" and child.text != "many":
            token_text = child.lemma_ + " " + token_text
    return token_text


def get_root_phrase(token, keywords):

    # xcomp: An open clausal complement (xcomp) of a verb or an adjective is a predicative or clausal complement
    #        without its own subject.

    for child in token.children:
        if child.dep_ == "acomp" or child.dep_ == "xcomp" or child.dep_ == "ccomp":
            keywords.append(child.lemma_)
    return keywords


def this_is_noun(tag_):
    # If is Noun/Proper Noun, be it Singular or Plural
    return tag_ == "NN" or tag_ == "NNP" or tag_ == "NNPS" or tag_ == "NNS"


def get_noun_chunk(sentence, en_doc, keywords):

    root_word = ""

    for token in sentence:

        # If the Noun itself is not a compound Noun then we can find its compound Nouns
        print(token)
        print("+sentence")
        if(this_is_noun(token.tag_)) and token.dep_ != "compound":
            token_text = get_compound_nouns(en_doc, token, token.text)
            keywords.append(token_text)

        if token.tag_ == "JJ" and token.dep_ == "attr":
            token_text = get_compound_nouns(en_doc, token, token.text)
            token_text = get_adj_phrase(token, token_text)
            keywords.append(token_text)

        # If is a Cardinal Number & dependency is numeric modifier
        # nummod : A numeric modifier of a noun is any number phrase that
        #          serves to modify the meaning of the noun with a quantity.
        if token.dep_ == "nummod" or token.tag_ == "CD":
            token_text = token.text

            if token.i > 0 and en_doc[token.i - 1].tag_ == "JJ":
                # If previous token is Adjective, the adjective is liked with the cardinal number
                token_text = en_doc[token.i - 1].text + " " + token.text

            if token.i < len(en_doc) - 1 and en_doc[token.i + 1].tag_ == "JJ":
                # If next token is Adjective
                token_text = token.text + " " + en_doc[token.i + 1].text

            keywords.append(token_text)

        # Extracts the root word of sentence
        if token.dep_ == "ROOT":
            root_word = token.lemma_
            print("++sentence")
            print(token)
            print("++sentence")
            keywords = get_root_phrase(token, keywords)

    return root_word, keywords


def extract_features(question_type, en_doc, show_debug=False):

    # NOTE: In the whole pipeline question_type is not used anywhere currently...

    keywords = []

    for sentence in en_doc.sents:
      print(sentence)
      print("+sentence")
      if show_debug:
        get_detail(sentence)
      root, keywords = get_noun_chunk(sentence, en_doc, keywords)
      keywords.append(root)

    return keywords



In [33]:
question = "What is age of sourav?"
en_doc_l = nlp(u'' + question)
print("Extracted: {0}".format(extract_features("", en_doc_l, True)))

What is age of sourav?
+sentence
What -- Lemma:what, Tag:WP, EntType:, Dep:nsubj, Head:is
is -- Lemma:be, Tag:VBZ, EntType:, Dep:ROOT, Head:is
age -- Lemma:age, Tag:NN, EntType:, Dep:nsubj, Head:is
of -- Lemma:of, Tag:IN, EntType:, Dep:prep, Head:age
sourav -- Lemma:sourav, Tag:NNS, EntType:, Dep:pobj, Head:of
? -- Lemma:?, Tag:., EntType:, Dep:punct, Head:is
What
+sentence
is
+sentence
++sentence
is
++sentence
age
+sentence
Compound Noun:age DEP nsubj
of
+sentence
sourav
+sentence
Compound Noun:sourav DEP pobj
?
+sentence
Extracted: ['age', 'sourav', 'be']


In [34]:
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


### **transform_vec**


In [35]:
question = "What is age of sourav kumar"
en_doc_l = nlp(u'' + question)
features = extract_features("", en_doc_l, False)
print(features)

What is age of sourav kumar
+sentence
What
+sentence
is
+sentence
++sentence
is
++sentence
age
+sentence
Compound Noun:age DEP attr
of
+sentence
sourav
+sentence
kumar
+sentence
Compound Noun:kumar DEP pobj
['age', 'sourav kumar', 'be']


In [36]:
import re
#print([word.split() for word in features])
ques = [word.split() for word in features]

mix_ques = ([[x,y] for x in ques for y in ques if x != y])

#listToString = str(','.join([str(elem) for elem in mix_ques]) + ","+ question +","+ ','.join([str(elem) for elem in ques])) 
 
print([' '.join([str(c) for c in lst]) for lst in mix_ques])


["['age'] ['sourav', 'kumar']", "['age'] ['be']", "['sourav', 'kumar'] ['age']", "['sourav', 'kumar'] ['be']", "['be'] ['age']", "['be'] ['sourav', 'kumar']"]


In [41]:
# input list 
l = mix_ques
 

# output list 
def lsToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s: 
      if type(ele) == list:
        str1 += " " +lsToString(ele)
      else: 
        str1 += " " +ele   
    
    # return string   
    return str1 
  
# function used for removing nested  
# lists in python.  
def removNestings(l): 
    output = [] 
    for i in l: 
        if type(i) == list: 
           output.append(removNestings(i)) 
        else: 
            output.append(i) 
    return output
  

def keywordMaker(l): 
    result = []
    for i in l:    
      result.append(lsToString(removNestings(i)))      
    return result
  

ques_Maker = keywordMaker(l)
ques_Maker.append(question) 
print(ques_Maker + ques)
 


['  age  sourav kumar', '  age  be', '  sourav kumar  age', '  sourav kumar  be', '  be  age', '  be  sourav kumar', 'What is age of sourav kumar', ['age'], ['sourav', 'kumar'], ['be']]


### temp

In [0]:
query_corpus = query2vec(["What is my age.","age","24","name"], dictionary)

In [0]:
print(corpus)

In [0]:
corpus_lsidf, query_lsidf = transform_vec(corpus, query_corpus)

In [0]:
print(corpus_lsidf)

In [0]:
simi_sorted = similarity(corpus_lsidf, query_lsidf)

In [0]:
print(len(simi_sorted))

In [0]:
    
    print("simi_sorted \n")
    print(simi_sorted)
    candidate_ans = []
    
    ##New Correction
    for sent in simi_sorted:
        sent_id = sent[0]
        if sent[1] > 0:
          candidate_ans.append(str(sentences[sent_id]))
          print(candidate_ans)
        
        
        

In [0]:
 from gensim.corpora import Dictionary
dct = Dictionary(["máma mele maso".split(), "ema má máma".split()])
ch = dct.doc2bow(["this", "is", "máma", "máma"])
print(dct)
print(ch)



In [0]:
dct.doc2bow(["this", "is", "máma"], return_missing=True)
([(2, 1)], {u'this': 1, u'is': 1})

### **dependency parse**

In [0]:
  print(candidate_ans)

In [0]:
doc = nlp('My age is 24. My father name is Uttam Kumar gupta. My name is Sourav and I am from India.')
 
for token in doc:
    print("{0}/{1} <--{2}-- {3}/{4}".format(
        token.text, token.tag_, token.dep_, token.head.text, token.head.tag_))

In [0]:
spacy.explain("NNP")

In [0]:
import spacy
from spacy import displacy
 
#displacy.serve(doc, style="dep")

In [0]:
import numpy as np
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from collections import Counter


#input text file
readFlNm="input.txt"
#precentage of summarization
percentageOfSummary=70
#fetch stop words
stopWords = set(stopwords.words('english'))
#update stop words
stopWords.update(['"', "'", ':', '(', ')', '[', ']', '{', '}']) #'.',  ',', '?', '!', ';'

#using pos_tag get the tag of words
def getTagsForWords(textLn2):
    tokens=word_tokenize(textLn2)
    tagged=pos_tag(tokens)
    return(tagged)

#remove stop words
def remStopWordsOur(lineIn):
    stopWords= {'i','a','and','about','an','are','as','at','be','by','com','for','from','how','in','is','it','not','of','on','or','that','the','this','to','was','what','when','where','who','will','with','the','www','your','is','am','some','you','your','I','A','And','About','An','Are','As','At','Be','By','Com','For','From','How','In','Is','It','Not','Of','On','Or','That','The','This','To','Was','What','When','Where','Who','Will','With','The','Www','Your','Is','Am','Some','You','Your','Was'}
    rmdStopWordsLn = ' '.join(w for w in lineIn.split() if w.lower() not in stopWords)
    return rmdStopWordsLn

#preprocessing the text and remove special characters
def preprocessText(lineIn):
    lineInLower=lineIn.lower()
    lineInRmdSplChars=lineInLower.replace('.',' ').replace(';',' ').replace(',',' ').replace('?',' ').replace('!',' ').replace(':',' ')
    return lineInRmdSplChars

#Divide the given text into lines
def getAllLines(lineIn):
    lineInReplcByPeriod=lineIn.replace('.','.§').replace(';',';§').replace(',',',§').replace('?','?§').replace('!','!§').replace('\n','§')
    linesOriginal=lineInReplcByPeriod.split('§')
    linesOriginal2=[item for item in linesOriginal if len(item)>0 ]
    return linesOriginal2

#Identify the noun position
def getNounPositions(type,tagged):
    nounPosi={}
    for item in tagged:
        if item[1]==type:
            nounPosi[item[0]]=-1
    
    for key in nounPosi.keys():
        regExpression=r'\b'+key.lower()+r'\b'
        nounsi=[m.start() for m in re.finditer(regExpression, lineIn.lower())]
#        print(key,nounsi)
        nounPosi[key]=nounsi
    return nounPosi
#Identify the pronoun position
def getProNounPositions(tagged):
    proNounPosi={}
    for item in tagged:
        if item[1]=='PRP': #or item[1]=='PRP$':
            proNounPosi[item[0].lower()]=-1
    
    for key in proNounPosi.keys():
        regExpression=r'\b'+key.lower()+r'\b'
        pronounsi=[m.start() for m in re.finditer(regExpression, lineIn.lower())]
#        print(key,pronounsi)
        proNounPosi[key]=pronounsi
    return proNounPosi

#Obtain nearest previous noun
def getNearestPreviousNoun(NNP,posiOfPronoun):
#    print('\t',NNP)    
    minimumDiff=len(lineIn)
    nearKey=''
    for keyNNP in NNP.keys():
        for posNoun in NNP[keyNNP]:
            if(posiOfPronoun>posNoun):
#                print('\t',posiOfPronoun-posNoun)
                if(minimumDiff>(posiOfPronoun-posNoun)):
                    minimumDiff=posiOfPronoun-posNoun
                    nearKey=keyNNP
#    print('\t near key=',nearKey)
    return nearKey

#Replace pronoun by noun
def pronounReplaceWithNearNoun(lineIn,PRP,NNP):
    replacePRP=[]       
    for key in PRP.keys():            
        for pos in PRP[key]:
            print('---------',key,'------',pos ,'-----')
            nearNoun=getNearestPreviousNoun(NNP,pos)
            replacePRP.append((key,pos,nearNoun))  
#    print(PRP)
#    print(replacePRP)
    
    replacePRP=sorted(replacePRP,key=lambda x:(-x[1],x[0],x[2]))
    lineInReplacePronn=lineIn
    for prpRep in replacePRP:
        lineInReplacePronn=lineInReplacePronn[:prpRep[1]]+prpRep[2]+lineInReplacePronn[prpRep[1]+len(prpRep[0]):]
    return lineInReplacePronn

#Based on weightage obtain the priority of lines
def obtainPriorotyOfALine(wtForLine):
    orderdLinesByWt=np.argsort(wtForLine)
    orderdLinesByWt=orderdLinesByWt[::-1]
    priority=[0]*len(wtForLine)
    
    for i in range(len(wtForLine)):
#        print(i,wtForLine[i],orderdLinesByWt[i])
        priority[orderdLinesByWt[i]]=i
    
    sentWtAndPriority=[]
    
    for i in range(len(wtForLine)):
        sentWtAndPriority.append((wtForLine[i],priority[i]))
    
    return sentWtAndPriority
#Construct summary by extraction method
def obtainSummary(lineForCalc,lineForExtract,percentageOfSummary):
    wtForLine=[0]*len(lineForCalc)
    print('Calcualting wt for lines......')
    for li in range(len(lineForCalc)):
    #    print('\t'+linesOriginal2[li])
        wtForLn=0.0
        preproccdLn2=preprocessText(lineForCalc[li])
        wInL=preproccdLn2.split()
        for w in wInL:
            w=preprocessText(w)
            if w in list(freqOfWords.keys()):
    #            print('\t\t'+w+' '+str(freqOfWords[w]))
                wtForLn=wtForLn+freqOfWords[w]
        wtForLine[li]=(wtForLn/len(wInL))
    #        print('\t\t'+lineForCalc[li]+' $'+str(wtForLine[li]))
    
    sentWtAndPriority=obtainPriorotyOfALine(wtForLine)
    print(sentWtAndPriority)
    numOfLinesInSummary=int((percentageOfSummary*len(lineForCalc))/100)
    reducedSummary=[]
    for li in range(len(lineForExtract)):
        if(sentWtAndPriority[li][1]<numOfLinesInSummary):
    #            print(li,sentWtAndPriority[li])
            reducedSummary.append(lineForExtract[li])
    return reducedSummary

#remove week nm and month name
def removeWeekNmMonthNm(NNP):
    entries = ('january','february','march','april','may','june','july','august','september','october','november','december','monday','tuesday','wednesday','thursday','friday','saturday','sunday')
    delKeys=[]
    for key in NNP.keys():
        if key.lower() in entries:
            print(key)
            delKeys.append(key)
    
    for key in delKeys:
        del NNP[key]
    return NNP


In [0]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from gensim.summarization.summarizer import summarize

In [0]:
pip install gensim

In [0]:
lineIn = "My name is Sourav and I am from India. My age is 24. I am currently stying in Passau. I am studing Mobile and embedded system. My father name is Uttam Kumar gupta. He is a business man. He own a shop. His work is of Photography. "
print( gensim.summarization.summarizer.summarize(lineIn, ratio=0.5, word_count=None, split=False))

In [0]:


lineIn = "My age is 24. My father name is Uttam Kumar gupta. My name is Sourav and I am from India."
print( gensim.summarization.summarizer.summarize(lineIn,ratio=0.5, word_count=None, split=False))
nt=len(lineIn.split())
freqOfWords = Counter(re.split(r'\s+',re.sub(r'[.,;\-!?]','',lineIn)))
for word, freq in freqOfWords.items():   
  freqOfWords[word]=freqOfWords[word]/nt

tagged=getTagsForWords(lineIn)
NNP=getNounPositions('NNP',tagged)
NNP=removeWeekNmMonthNm(NNP)
#NN=getNounPositions('NN',tagged)
#NNS=getNounPositions('NNS',tagged)
PRP=getProNounPositions(tagged)

linesOriginal2=getAllLines(lineIn)
lineInReplacePronn=pronounReplaceWithNearNoun(lineIn,PRP,NNP)
linesReplacedPronn2=getAllLines(lineInReplacePronn)
print(linesReplacedPronn2)
print(percentageOfSummary)

#perform the text summarization without pronoun replacement
reducedSummaryWithoutReplc=obtainSummary(linesOriginal2,linesOriginal2,percentageOfSummary)
#perform the text summarization with pronoun replacement
reducedSummaryWithReplc=obtainSummary(linesReplacedPronn2,linesOriginal2,percentageOfSummary)


#text summeriazation.

print(reducedSummaryWithReplc)


In [0]:
doc = nlp(lineIn)

from prettytable import PrettyTable
t = PrettyTable(['text', 'token.head.text','token.head.tag_','child','pos_','tag_','dep_','shape_',])
 
for token in doc:
  child =  ([child for child in token.children])
  t.add_row([token.text,token.head.text, spacy.explain(token.head.tag_), str(child), spacy.explain(token.pos_),spacy.explain(token.tag_) ,spacy.explain(token.dep_) ,
          spacy.explain(token.shape_)])
    
print(t)

In [0]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [0]:
root = [token for token in doc if token.head == token][0]
subject = list(root.lefts)[0]
for descendant in subject.subtree:
    assert subject is descendant or subject.is_ancestor(descendant)
    print(descendant.text, descendant.dep_, descendant.n_lefts,
            descendant.n_rights,
            [ancestor.text for ancestor in descendant.ancestors])

### **Sentiment**


In [0]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [0]:
nltk.download('vader_lexicon')

In [0]:
 tricky_sentences = [
    "i really liked the movie and had fun. Sentiment",
    "VADER sentiment analysis is the shit.",
    "Sentiment analysis has never been good.",
    "Sentiment analysis with VADER has never been this good.",
    "Warren Beatty has never been so entertaining.",
    "I won't say that the movie is astounding and I wouldn't claim that \
    the movie is too banal either.",
    "I like to hate Michael Bay films, but I couldn't fault this one",
    "It's one thing to watch an Uwe Boll film, but another thing entirely \
    to pay for it",
    "The movie was too good",
    "This movie was actually neither that funny, nor super witty.",
    "This movie doesn't care about cleverness, wit or any other kind of \
    intelligent humor.",
    "Those who find ugly meanings in beautiful things are corrupt without \
    being charming.",
    "There are slow and repetitive parts, BUT it has just enough spice to \
    keep it interesting.",
    "The script is not fantastic, but the acting is decent and the cinematography \
    is EXCELLENT!",
    "Roger Dodger is one of the most compelling variations on this theme.",
    "Roger Dodger is one of the least compelling variations on this theme.",
    "Roger Dodger is at least compelling as a variation on the theme.",
    "they fall in love with the product",
    "but then it breaks",
    "usually around the time the 90 day warranty expires",
    "this is  excellent",
    "However, Mr. Carter solemnly argues, his client carried out the kidnapping \
    under orders and in the ''least offensive way possible.''"
 ]
 sentences.extend(tricky_sentences)
 sid = SentimentIntensityAnalyzer()
 for sentence in tricky_sentences :
     print(sentence)
     ss = sid.polarity_scores(sentence)
     for k in sorted(ss):
         print('{0}: {1}, '.format(k, ss[k]), end='')
     print()

### **Keras**

In [0]:
from numpy import array
from keras.preprocessing.text import one_hot
docs = ['Gut gemacht',
        'Gute arbeit',
        'Super idee',
        'Perfekt erledigt',
        'exzellent',
        'naja',
        'Schwache arbeit.',
        'Nicht gut',
        'Miese arbeit.',
        'Hätte es besser machen können.']
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)